In [1]:
import os
import re
import subprocess
from collections import defaultdict
import numpy as np
import rasterio
from rasterio.merge import merge
from rasterio.warp import reproject, Resampling
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import glob
import json
from tqdm import tqdm
import yaml

#import geopandas as gpd
import pandas as pd

import torch

import sys
sys.path.append('../')

import utils.basics as bsc 
import utils.plotting as pt
import utils.processing as proc
import utils.eval_pipe as eval

import utils.model_loader as md
import utils.data_loader as dt
import utils.config_loader as cf

%load_ext autoreload
%autoreload 2


## main run of experiments

In [4]:
# SELECT EXPERIMENTAL CONFIG
with open('../configs/experiments.yaml', 'r') as f:
    experiments = yaml.safe_load(f)
    experiment_names = list(experiments.keys())
    experiment_names = experiment_names[1:8]  # Select the first 6 experiment names
    #experiment_names = ['01_baseline',
    #                    '02_pixel_composites',
    #                    '03_seasonal_composites',
    #                    '04_pixel_and_seasonal_comp',
    #                    '05_support_fmask_dlt',
    #                      '06_support_fmask_dlt_dem',
                        #'07_aux_task',
                       # ]
    
with open('../configs/normparams.yaml', 'r') as f:
    normparams = yaml.safe_load(f)
joint_normparams = normparams['chm']['_111']
#print(experiment_names)
experiment_names
sites, cfg = cf.get_config('01_baseline')  
combos = ["110","101","011"] 


In [5]:
experiment_names

['01_baseline',
 '02_pixel_composites',
 '03_seasonal_composites',
 '04_pixel_and_seasonal_comp',
 '05_support_fmask_dlt',
 '06_support_fmask_dlt_dem',
 '07_aux_task']

In [ ]:
import random

global_config = md.global_config
# reproducible shuffling (num_workers=0 -> simpler)
seed = global_config['seed'] 
run_id_base = "251024_GEN2_"
repetitions = range(2)
combos = ["110","101","011"] #1 is training data, 0 test. logic is LSB: 001 -> SITE1 is training data. 

for i in repetitions: # Run 10 experiments with different seeds

    for combo in combos:
        #run_id = md.generate_run_id()
        run_id = run_id_base + f"_{combo}_{i}"
        seed = seed + i  # Different seed for each experiment
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)

        for exp_name in experiment_names:
            
            sites, cfg = cf.get_config(exp_name)  
            cfg.update(global_config)  # Ensure cfg has the latest global_config
            cfg.update({"combo": combo})
            #print("=== NEW EXPERIMENT ===")
            print(f" --> Name: {exp_name}, Combo: {combo}, Run ID: {run_id}, Seed: {seed}")

            # Build dataset
            #X, Y = dt.build_patched_dataset(cfg, sites, patch_size=32, nan_percent_allowed=20)
            X_patch_train, Y_patch_train, X_test, Y_test = dt.build_patched_dataset_generalization(cfg, sites, combo,patch_size=32, nan_percent_allowed=20)
            
            # now split the train data into train (80%) and val (20%)
            X_train, X_val, Y_train, Y_val = train_test_split(
                X_patch_train, Y_patch_train, test_size=0.2, random_state=seed
            )

            # split into train/val/test (70/15/15) with your logic
            #(X_train, Y_train), (X_val, Y_val), (X_test, Y_test) = dt.split_dataset(X, Y, seed=seed)

            # Syntax: X_train, X_val, X_test, y_train, y_val, y_test 
            train_dataset = md.S2CanopyHeightDataset(X_train, Y_train, cfg)
            val_dataset = md.S2CanopyHeightDataset(X_val, Y_val, cfg)
            test_dataset = md.S2CanopyHeightDataset(X_test, Y_test, cfg)

            train_loader = md.DataLoader(
                train_dataset,
                batch_size=global_config['batch_size'],
                shuffle=True,
                num_workers=0
            )
            val_loader = md.DataLoader(val_dataset, batch_size=global_config['batch_size'])
            test_loader = md.DataLoader(test_dataset, batch_size=global_config['batch_size'])
            
            if False:
                print(f"Train dataset valid pixel share: \t{train_dataset.validshare():.4f}, count: {len(train_dataset)}")
                print(f"Validation dataset valid pixel share: \t{val_dataset.validshare():.4f}, count: {len(val_dataset)}")
                print(f"Test dataset valid pixel share: \t{test_dataset.validshare():.4f}, count: {len(test_dataset)}")
            
            # build model depending on in out channels, defined by the dataloaders
            model = md.build_unet(in_channels=X_train[0].shape[0], out_channels=Y_train[0].shape[0])
            #train model depending on config. 
            model, logs = md.train_model(model, train_loader, val_loader, cfg, md.global_config)

            # evaluate model on val and test set, save results
            # get normparams for CHMmmn
            combo_key = f"_{combo}"
            normparams_used = normparams['chm'][combo_key]

            md.save_results(model, val_loader, test_loader, normparams_used, logs, cfg, run_id=run_id)

            #print("=================")

        #print("DONE WITH COMBO ", combo)
    print("DONE WITH ALL EXPERIMENTS for iteration:  ", i)



=== NEW EXPERIMENT ===
 --> Name: 01_baseline, Combo: 110, Run ID: 251021_GEN__110_2, Seed: 52


Epochs: 100%|██████████| 500/500 [04:45<00:00,  1.75it/s]


Results saved to: ../results/251021_GEN__110_2/train/01_baseline
=== NEW EXPERIMENT ===
 --> Name: 02_pixel_composites, Combo: 110, Run ID: 251021_GEN__110_2, Seed: 52


Epochs: 100%|██████████| 500/500 [05:23<00:00,  1.55it/s]


Results saved to: ../results/251021_GEN__110_2/train/02_pixel_composites
=== NEW EXPERIMENT ===
 --> Name: 03_seasonal_composites, Combo: 110, Run ID: 251021_GEN__110_2, Seed: 52


...

Results saved to: ../results/251021_GEN__011_4/train/05_support_fmask_dlt
=== NEW EXPERIMENT ===
 --> Name: 06_support_fmask_dlt_dem, Combo: 011, Run ID: 251021_GEN__011_4, Seed: 77


Epochs: 100%|██████████| 500/500 [04:38<00:00,  1.79it/s]


Results saved to: ../results/251021_GEN__011_4/train/06_support_fmask_dlt_dem
=== NEW EXPERIMENT ===
 --> Name: 07_aux_task, Combo: 011, Run ID: 251021_GEN__011_4, Seed: 77


Epochs: 100%|██████████| 500/500 [04:41<00:00,  1.78it/s]


Results saved to: ../results/251021_GEN__011_4/train/07_aux_task
DONE WITH COMBO  011
DONE WITH ALL EXPERIMENTS for iteration:   4


In [46]:
combo_key = f"_{110}"

normparams_used = normparams['chm'][combo_key]

In [47]:
normparams_used

{'mu': 28.23365592956543, 'n': 755154, 'std': 5.794953346252441}

In [52]:
# fix the preds and save them again
experiment_names = ["02_pixel_composites"]
global_config = md.global_config
# reproducible shuffling (num_workers=0 -> simpler)
seed = global_config['seed']
run_id_base = "251021_GEN_"
repetitions = 2 # only the first rep was done so far
combos = ["110","101","011"] #1 is training data, 0 test. logic is LSB: 001 -> SITE1 is training data. 

for i in range(repetitions): # Run 10 experiments with different seeds

    for combo in combos:
        #run_id = md.generate_run_id()
        run_id = run_id_base + f"_{combo}_{i}"
        seed = seed + i  # Different seed for each experiment
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)

        for exp_name in experiment_names:
            
            sites, cfg = cf.get_config(exp_name)  
            cfg.update(global_config)  # Ensure cfg has the latest global_config
            cfg.update({"combo": combo})
            print("=== NEW EXPERIMENT ===")
            print(f" --> Name: {exp_name}, Combo: {combo}, Run ID: {run_id}, Seed: {seed}")

            # Build dataset
            #X, Y = dt.build_patched_dataset(cfg, sites, patch_size=32, nan_percent_allowed=20)
            X_patch_train, Y_patch_train, X_test, Y_test = dt.build_patched_dataset_generalization(cfg, sites, combo,patch_size=32, nan_percent_allowed=20)
            
            # now split the train data into train (80%) and val (20%)
            X_train, X_val, Y_train, Y_val = train_test_split(
                X_patch_train, Y_patch_train, test_size=0.2, random_state=seed
            )

            # split into train/val/test (70/15/15) with your logic
            #(X_train, Y_train), (X_val, Y_val), (X_test, Y_test) = dt.split_dataset(X, Y, seed=seed)

            # Syntax: X_train, X_val, X_test, y_train, y_val, y_test 
            #train_dataset = md.S2CanopyHeightDataset(X_train, Y_train, cfg)
            val_dataset = md.S2CanopyHeightDataset(X_val, Y_val, cfg)
            test_dataset = md.S2CanopyHeightDataset(X_test, Y_test, cfg)

            #train_loader = md.DataLoader( train_dataset, batch_size=global_config['batch_size'],shuffle=True, num_workers=0)
            val_loader = md.DataLoader(val_dataset, batch_size=global_config['batch_size'])
            test_loader = md.DataLoader(test_dataset, batch_size=global_config['batch_size'])
            
            if False:
                print(f"Train dataset valid pixel share: \t{train_dataset.validshare():.4f}, count: {len(train_dataset)}")
                print(f"Validation dataset valid pixel share: \t{val_dataset.validshare():.4f}, count: {len(val_dataset)}")
                print(f"Test dataset valid pixel share: \t{test_dataset.validshare():.4f}, count: {len(test_dataset)}")
            
            # build model depending on in out channels, defined by the dataloaders
            model = md.build_unet(in_channels=X_train[0].shape[0], out_channels=Y_train[0].shape[0])
            #train model depending on config. 
            #model, logs = md.train_model(model, train_loader, val_loader, cfg, md.global_config)

            out_dir = os.path.join("../results", run_id, 'train', exp_name)
            if not os.path.exists(out_dir):
                continue 
            model_weights, logs, cfg = md.load_results(exp_name, run_id)
            model.load_state_dict(model_weights) # fix the preds and save them again
            # evaluate model on val and test set, save results
            # get normparams for CHM
            combo_key = f"_{combo}"
            normparams_used = normparams['chm'][combo_key]

            md.save_results(model, val_loader, test_loader, normparams_used, logs, cfg, run_id=run_id)

            print("=================")

        print("DONE WITH COMBO ", combo)
    print("DONE WITH ALL EXPERIMENTS for iteration:  ", i)


=== NEW EXPERIMENT ===
 --> Name: 02_pixel_composites, Combo: 110, Run ID: 251021_GEN__110_0, Seed: 47
Results saved to: ../results/251021_GEN__110_0/train/02_pixel_composites
DONE WITH COMBO  110
=== NEW EXPERIMENT ===
 --> Name: 02_pixel_composites, Combo: 101, Run ID: 251021_GEN__101_0, Seed: 47
Results saved to: ../results/251021_GEN__101_0/train/02_pixel_composites
DONE WITH COMBO  101
=== NEW EXPERIMENT ===
 --> Name: 02_pixel_composites, Combo: 011, Run ID: 251021_GEN__011_0, Seed: 47
Results saved to: ../results/251021_GEN__011_0/train/02_pixel_composites
DONE WITH COMBO  011
DONE WITH ALL EXPERIMENTS for iteration:   0
=== NEW EXPERIMENT ===
 --> Name: 02_pixel_composites, Combo: 110, Run ID: 251021_GEN__110_1, Seed: 48
Results saved to: ../results/251021_GEN__110_1/train/02_pixel_composites
DONE WITH COMBO  110
=== NEW EXPERIMENT ===
 --> Name: 02_pixel_composites, Combo: 101, Run ID: 251021_GEN__101_1, Seed: 49
DONE WITH COMBO  101
=== NEW EXPERIMENT ===
 --> Name: 02_pixel_